# Regressão Linear com Rede Neural

<a target="_blank" href="https://colab.research.google.com/github/silash35/labrob/blob/master/Others/PyTorch/src/NN.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 1. Importar as bibliotecas necessárias.

A Matplotlib vai nos ajudar a criar gráficos e visualizar o que está acontecendo.
Já o PyTorch é a biblioteca responsável por criar efetivamente a rede neural, e nos poupar de implementar vários algoritmos úteis manualmente.

In [1]:
from matplotlib import pyplot as plt
import torch
from torch import nn